In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch, os, faiss
from FlagEmbedding import FlagModel
import numpy as np

In [ ]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://104.197.136.212:8877/v1",
    api_key="token-abc123",
)

In [ ]:
'''
import datasets

wikidata = datasets.load_dataset('yongchanskii/wiki-korean')
wikidata = wikidata.remove_columns(['id', 'url'])

In [ ]:
#embedding_text = wikidata['train'].select(range(5500,6000))

In [ ]:
'''
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents(embedding_text['text'])

In [ ]:
index = faiss.read_index("index_faiss")

In [10]:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [11]:
model = FlagModel('BAAI/bge-m3',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

In [27]:
question = "2023 대한민국 FA컵 결과 알려줘."

In [28]:
queries = [question,]

rag_docs = ''
query_embeddings = model.encode_queries(queries)

dists, ids = index.search(query_embeddings.astype(np.float32), k=3)

print(f"query:\t{queries[0]}\n\n")
count = 0
for i in range(3):
    rag_docs += f"후보문장_{count}:\t{texts[ids[0][i]]}\n"
    count +=1

print(rag_docs)

query:	2023 대한민국 FA컵 결과 알려줘.


후보문장_0:	page_content='외부 링크'
후보문장_1:	page_content='역대 선거 결과

각주

외부 링크 
 김동길 교수의 Freedom Watch
 
 조인스 데이터 - 산남 김동길(山南 金東吉) 前 연세대학교 부총장
 두산대백과사전 - 산남 김동길(山南 金東吉) 前 통일국민당 국회의원
 보트 비전 - 산남 김동길(山南 金東吉) 前 자유민주연합 국회의원
 두산대백과사전 - 산남 김동길(山南 金東吉) 前 무소속 국회의원
 김동길TV https://www.youtube.com/channel/UCCCzHfAALDPQ0L11UTsKTWA'
후보문장_2:	page_content='스포츠

축구 
노르트라인베스트팔렌 주는 여러 축구 클럽이 있다: 1. FC 쾰른, 바이어 04 레버쿠젠, 보루시아 도르트문트, 보루시아 묀헨글라트바흐, 아르미니아 빌레펠트, 알레마니아 아헨, 부퍼탈 SV 보루시아, 포르투나 뒤셀도르프, FC 샬케 04, MSV 뒤스부르크, SC 파더보른 07, VfL 보훔

또한 이 주에서 1974년 FIFA 월드컵과 2006년 FIFA 월드컵의 경기가 열렸으며, 2011년 FIFA 여자 월드컵의 경기도 개최했다. 1974년 월드컵에서는 뒤셀도르프에 있는 라인슈타디온, 겔젠키르헨의 파르크슈타디온에서 경기가 열렸으며, 2006년 월드컵에서는 쾰른의 라인에네르기슈타디온, 도르트문트의 지그날 이두나 파르크, 겔젠키르헨의 펠틴스 아레나에서 열렸다. 레버쿠젠의 바이아레나와 보훔에 있는 루르슈타디온이 2011년 여자 FIFA 월드컵의 개최 경기장이 되었다.'



In [29]:
system_prompt = """You are a helpful assistant. And Answers must be in Korean. 
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘. 
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요. 
2) 그리고 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.
3) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

user_prompt = question + "\n후보문장: " + rag_docs

In [15]:
user_prompt

'석기시대란 무엇인가?\n후보문장: 후보문장_0:\tpage_content=\'석기 시대(石器時代, )는 인류 역사를 추적하는 데 고고학 및 역사학에서 쓰이는 낱말로서 인류, 정확히는 인류의 조상인 원인이 돌로 만든 도구를 쓰기 시작한 시대를 일컫는다.\n\n석기는 다양한 종류의 돌로 만든다. 부싯돌과 규질암은 자르는 도구나 무기로 만들어졌고, 현무암이나 사암은 맷돌과 같이 가는 도구로 만들어졌다. 나무, 뼈, 조개껍질, 사슴의 뿔, 그리고 다른 재료들도 필요에 따라 만들어 쓴 것으로 보인다. 이 시기의 말기에는 진흙과 같은 점토도 도기를 만들기 위해 사용되었다. 일련의 기술적 진보가 이후, 동기 시대(Chalcolithic), 청동기 시대, 철기 시대처럼 특징화 된다.\'\n후보문장_1:\tpage_content=\'구석기 시대 \n석기시대(Paleolithic)는 "old"를 뜻하는 그리스어의 παλαιός, palaios와 "stone"을 뜻하는 λίθος, lithos의 합성어로 "고대 석기 시대"라는 의미로 1865년 고고학자 존 러벅이 만든 용어이다. 이 시기는 석기를 도구로 구분된 선사 시대이다. 지상 위의 인류가 99% 이상 이것을 사용하며, 250만년에서 260만년 전까지 거슬러 올라간다. 호모 하빌리스와 같은 고대 인류가 이러한 석기를 사용하며, 농경이 소개되고, 플라이스토세 말기인 기원전 10000년경에 끝난다. 플라이스토세 기는 중석기 시대나 초기 신석기화 또는 준석기 시대에 끝을 맺는다.\'\n후보문장_2:\tpage_content=\'새로운 고고학적 발견으로 새로운 시대와 세부 시대가 추가되어 서로 다른 지역의 상태를 설명하기 위해서 다른 도해가 개발되었다. 석기 시대의 보다 현대적인 시대구분은 구석기 시대에서 신석기 시대를 다음과 같이 분류를 한다.\n 플라이스토세 기(홍적세 또는 갱신세) - 두터운 빙하기\n 구석기 시대 (Paleolithic)\n 홀로세 기(충적세 또는 현세) - 현대적인 날씨\n 중석기 시대 (Mesolithi

In [30]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
assistant_response = generate_response(messages)

2023 대한민국 FA컵 결과를 알려드릴 수 없습니다. 좀 더 구체적으로 질문해 주세요.

적합한 후보문장이 없습니다.

inference time: 1.19914 sec 




In [16]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "Llama-3.1-8B-Instruct",
        messages = message_list,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )
    
    final_answer = []
    assistant_response = ""
    
    start = time.time()
    
    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content
        
        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content
    
    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response